Pressure Loss calculation

In [ ]:
from lib.constants import Calculations
from lib.topdrive import vel_acc
from lib.circulation import circl
from scipy.integrate import solve_ivp
import numpy as np

clc = Calculations(depth=13000)
num_t_step = int(clc.run_time / 0.1)  # time step size
clc.run_time
times = np.linspace(0, clc.run_time, num_t_step)
forces_over_time = np.zeros((int(num_t_step), len(clc.MD)))  # Store force profiles over time

In [ ]:
from lib.constants import Calculations
from lib.topdrive import vel_acc
from lib.circulation import circl
from scipy.integrate import solve_ivp
import numpy as np


def trip_operation(F_b, flowrate, length, operation='POOH'):
    clc = Calculations(depth=length)
    num_t_step = int(clc.run_time / 0.1)  # time step size
    times = np.linspace(0, clc.run_time, num_t_step)
    forces_over_time = np.zeros((num_t_step, len(clc.MD)))  # Store force profiles over time

    for idx, time in enumerate(times):
        fluid_force = circl(flowrate, length) if flowrate != 0 else np.zeros_like(clc.MD[:-1])

        def dFds(s, F):
            # Current velocity and acceleration
            velocity, acceleration = vel_acc(time)
            
            # Interpolate spatial parameters
            DLS_interp = np.interp(s, clc.MD[:-1], clc.DLS)
            n_z_interp = np.interp(s, clc.MD[:-1], clc.n_z)
            b_z_interp = np.interp(s, clc.MD[:-1], clc.b_z)
            t_z_interp = np.interp(s, clc.MD[:-1], clc.t_z)
            bw_pipe_interp = np.interp(s, clc.MD[:-1], clc.bw_pipe)
            fluid_interp = np.interp(s, clc.MD[:-1], fluid_force)
            velocity_interp = np.interp(s, clc.MD[:-1], velocity)
            acceleration_interp = np.interp(s, clc.MD[:-1], acceleration)
            

            # Additional terms
            term1 = np.pi/4*(clc.global_od_array**2 - clc.global_id_array**2)*clc.pipe_density*velocity_interp**2
            term2 = np.pi/4*(clc.global_od_array**2 - clc.global_id_array**2)*clc.pipe_density*acceleration_interp
            
            # Calculate the contact force
            # w_c = np.sqrt((F * DLS_interp + bw_pipe_interp * n_z_interp)**2 + (bw_pipe_interp * b_z_interp)**2)

            if operation == 'POOH':
                w_c = np.sqrt((F * DLS_interp + bw_pipe_interp * n_z_interp - term1)**2 + (bw_pipe_interp * b_z_interp)**2)
                return -(bw_pipe_interp*t_z_interp + clc.ff*w_c + fluid_interp + term2)
            elif operation == 'RIH':
                w_c = np.sqrt((F * DLS_interp + bw_pipe_interp * n_z_interp + term1)**2 + (bw_pipe_interp * b_z_interp)**2)
                return -(bw_pipe_interp*t_z_interp - clc.ff*w_c + fluid_interp + term2)
            elif operation == 'ROB':
                return -(bw_pipe_interp*t_z_interp + fluid_interp)
            else:
                raise ValueError("Unknown operation type specified.")

        # Spatial integration
        s_span = [clc.MD[-1], clc.MD[0]]  # from bottom to top
        solution = solve_ivp(dFds, s_span, [F_b], method='RK45', t_eval=np.flip(clc.MD))

        # Store the computed force profile at this time step
        forces_over_time[idx] = solution.y.flatten()

    return times, forces_over_time


In [ ]:

# Call the function with the appropriate parameters
times, forces_over_time = trip_operation(F_b=0, flowrate=0, length=13000, operation='POOH')

In [ ]:
from lib.constants import Calculations
from lib.topdrive import vel_acc
from lib.circulation import circl
from scipy.integrate import solve_ivp
import numpy as np

length = 13000
flowrate = 0

clc = Calculations(depth=length)
fluid_force = circl(flowrate, length) if flowrate != 0 else np.zeros_like(clc.MD[:-1])

def dFds(s, F, velocity, acceleration):
    # Interpolate the necessary values for the current s
    DLS_interp = np.interp(s, clc.MD[:-1], clc.DLS)
    n_z_interp = np.interp(s, clc.MD[:-1], clc.n_z)
    b_z_interp = np.interp(s, clc.MD[:-1], clc.b_z)
    t_z_interp = np.interp(s, clc.MD[:-1], clc.t_z)
    bw_pipe_interp = np.interp(s, clc.MD[:-1], clc.bw_pipe)
    fluid_interp = np.interp(s, clc.MD[:-1], fluid_force)

    # Calculate the contact force with new dynamic terms
    w_c = np.sqrt((F * DLS_interp + bw_pipe_interp * n_z_interp - clc.pipe_density * np.pi*(clc.global_hole_array**2 - clc.global_od_array**2)/4 * velocity**2)**2 + (bw_pipe_interp * b_z_interp)**2)
    dynamic_term = clc.pipe_density * np.pi*(clc.global_hole_array**2 - clc.global_od_array**2)/4 * acceleration
    return -(bw_pipe_interp*t_z_interp + clc.ff*w_c + fluid_interp) - dynamic_term

def dFdt(F_initial, spatial_domain, velocities, accelerations):
    """ Solve the spatial domain at each time point """
    hookload_over_time = []
    for i, _ in enumerate(times):
        # Call the ODE solver for each time step with corresponding velocity and acceleration
        sol = solve_ivp(lambda s, F: dFds(s, F, velocities[i], accelerations[i]),
                        spatial_domain, F_initial, method='RK45')
        hookload_over_time.append(sol.y[-1][-1])  # Append the last value (force at the top)
    return hookload_over_time

# Spatial and time domains
spatial_domain = [clc.MD[-1], clc.MD[0]]
times = np.linspace(0, 10, 100)
F_initial = [0]

# Precompute or define velocity and acceleration functions
velocities, accelerations = vel_acc(times)

# Solve for each time step
hookload_over_time = dFdt(F_initial=F_initial, spatial_domain=spatial_domain, velocities=velocities, accelerations=accelerations)

print("Hookload values over time:", hookload_over_time)


New version BELOW

In [1]:
from lib.constants import Calculations
from lib.topdrive import vel_acc
from lib.circulation import circl
from lib.init_xls import get_area
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
import numpy as np

length = 13000
flowrate = 0

clc = Calculations(depth=length)
fluid_force = circl(flowrate, length) if flowrate != 0 else np.zeros_like(clc.MD[:-1])


c:\Users\elmir.hamidov\AppData\Local\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Web Extension extension is not supported and will be removed
  for idx, row in parser.parse():


In [4]:
def trip_operation(F_b, flowrate, length, operation='POOH'):
    clc = Calculations(depth=length)
    fluid_force = circl(flowrate, length) if flowrate != 0 else np.zeros_like(clc.MD[:-1])

    def dFds(s, F, velocity, acceleration):
        cross_area = np.pi*(clc.global_hole_array**2 - clc.global_od_array**2)/4
        
        # Interpolate the necessary values for the current s
        DLS_interp = np.interp(s, clc.MD[:-1], clc.DLS)
        n_z_interp = np.interp(s, clc.MD[:-1], clc.n_z)
        b_z_interp = np.interp(s, clc.MD[:-1], clc.b_z)
        t_z_interp = np.interp(s, clc.MD[:-1], clc.t_z)
        bw_pipe_interp = np.interp(s, clc.MD[:-1], clc.bw_pipe)
        fluid_interp = np.interp(s, clc.MD[:-1], fluid_force)

        # Calculate the contact force with new dynamic terms
        vel_term = 12/231*clc.pipe_density * get_area(s, clc.MD, cross_area) * velocity**2                       # Velocity term
        acc_term = 12/231*clc.pipe_density * get_area(s, clc.MD, cross_area) * acceleration                      # Acceleration term
        w_c = np.sqrt((F * DLS_interp + bw_pipe_interp * n_z_interp - vel_term)**2 + (bw_pipe_interp * b_z_interp)**2)

        if operation == 'POOH':
            return -(bw_pipe_interp*t_z_interp + clc.ff*w_c + fluid_interp + acc_term)
        elif operation == 'RIH':
            return -(bw_pipe_interp*t_z_interp - clc.ff*w_c + fluid_interp + acc_term)
        elif operation == 'ROB':
            return -(bw_pipe_interp*t_z_interp + fluid_interp)
        else:
            raise ValueError("Unknown operation type specified.")

    s_span = [clc.MD[-1], clc.MD[0]]  # Span from the end to the start
        
    if operation == 'POOH':
        times = np.arange(0, clc.a3, 0.5)           # Time domain
        vel, acc = vel_acc(times, clc)                   # Computing velocity and acceleration
        
    elif operation == 'RIH':
        times = np.arange(clc.a3, clc.a6, 0.5)      # Time domain
        vel, acc = vel_acc(times, clc)                   # Computing velocity and acceleration

    elif operation == 'ROB':
        times = np.arange(0, 30, 0.5)               # Time domain
        vel, acc = (
            np.zeros(len(times)), 
            np.zeros(len(times))
            )                                       # Computing velocity and acceleration
    else:
        raise ValueError("Unknown operation type specified.")
    
    sol = [solve_ivp(dFds, 
                    s_span, 
                    [F_b], 
                    args=(vel[i], acc[i]), 
                    method='RK45', 
                    t_eval=np.flip(clc.MD),
                    max_step=10,
                    rtol=0.001,
                    atol=0.001,
                    dense_output=True
                    ).y.flatten()[-1] for i,_ in enumerate(times)]
    
    return sol

In [59]:
# def dFds(s, F, velocity, acceleration):
#     cross_area = np.pi*(clc.global_od_array**2 - clc.global_id_array**2)/4
    
#     # Interpolate the necessary values for the current s
#     DLS_interp = np.interp(s, clc.MD[:-1], clc.DLS)
#     n_z_interp = np.interp(s, clc.MD[:-1], clc.n_z)
#     b_z_interp = np.interp(s, clc.MD[:-1], clc.b_z)
#     t_z_interp = np.interp(s, clc.MD[:-1], clc.t_z)
#     bw_pipe_interp = np.interp(s, clc.MD[:-1], clc.bw_pipe)
#     fluid_interp = np.interp(s, clc.MD[:-1], fluid_force)

#     # Calculate the contact force with new dynamic terms
#     vel_term = 12/231*clc.pipe_density * get_area(s, clc.MD, cross_area) * DLS_interp * velocity**2 # Velocity term
#     acc_term = 12/231*clc.pipe_density * get_area(s, clc.MD, cross_area) * acceleration # Acceleration term
#     w_c = np.sqrt((F * DLS_interp + bw_pipe_interp * n_z_interp - vel_term)**2 + (bw_pipe_interp * b_z_interp)**2)
#     return -(bw_pipe_interp*t_z_interp - clc.ff*w_c + fluid_interp + acc_term)

def dFds(s, F, velocity, acceleration, operation):
        cross_area = np.pi*(clc.global_od_array**2 - clc.global_id_array**2)/4
        
        # Interpolate the necessary values for the current s
        DLS_interp = np.interp(s, clc.MD[:-1], clc.DLS)
        n_z_interp = np.interp(s, clc.MD[:-1], clc.n_z)
        b_z_interp = np.interp(s, clc.MD[:-1], clc.b_z)
        t_z_interp = np.interp(s, clc.MD[:-1], clc.t_z)
        bw_pipe_interp = np.interp(s, clc.MD[:-1], clc.bw_pipe)
        fluid_interp = np.interp(s, clc.MD[:-1], fluid_force)

        # Calculate the contact force with new dynamic terms
        vel_term = 12/231*clc.pipe_density * get_area(s, clc.MD, cross_area) * velocity**2                       # Velocity term
        acc_term = 12/231*clc.pipe_density * get_area(s, clc.MD, cross_area) * acceleration                      # Acceleration term
        w_c = np.sqrt((F * DLS_interp + bw_pipe_interp * n_z_interp - vel_term)**2 + (bw_pipe_interp * b_z_interp)**2)

        if operation == 'POOH':
            return -bw_pipe_interp*t_z_interp + clc.ff*w_c + fluid_interp + acc_term
        elif operation == 'RIH':
            return -(bw_pipe_interp*t_z_interp - clc.ff*w_c + fluid_interp + acc_term)
        elif operation == 'ROB':
            return -(bw_pipe_interp*t_z_interp + fluid_interp)
        else:
            raise ValueError("Unknown operation type specified.")
        
def dFdt(F_b, operation):
    s_span = [clc.MD[-1], clc.MD[0]]  # Span from the end to the start
        
    if operation == 'POOH':
        times = np.arange(0, clc.a3, 0.5)           # Time domain
        vel, acc = vel_acc(times, clc)              # Computing velocity and acceleration
        
    elif operation == 'RIH':
        times = np.arange(clc.a3, clc.a6, 0.5)      # Time domain
        vel, acc = vel_acc(times, clc)              # Computing velocity and acceleration

    elif operation == 'ROB':
        times = np.arange(0, 30, 0.5)               # Time domain
        vel, acc = (
            np.zeros(len(times)), 
            np.zeros(len(times))
            )                                       # Computing velocity and acceleration
    else:
        raise ValueError("Unknown operation type specified.")
    
    sol = [solve_ivp(dFds, 
                    s_span, 
                    F_b, 
                    args=(vel[i], acc[i], operation), 
                    method='RK45', 
                    t_eval=np.flip(clc.MD),
                    max_step=10,
                    rtol=0.001,
                    atol=0.001,
                    dense_output=True
                    ).y.flatten()[-1] for i,_ in enumerate(times)]
    
    return sol, times


In [11]:
# Define parameters

# spatial_domain = [clc.MD[-1], clc.MD[0]]  # Spatial domain from bottom to top of the drillstring
# times = np.arange(37, 75, 0.5)  # Time domain from 0 to 10 seconds
# F_initial = [0]  # Initial condition for force at the bottom

# Precompute or define velocity and acceleration functions
# vel, acc = vel_acc(times)
# velocity, acceleration = lambda t: vel_acc(np.array([t])) # Define velocity_function and acceleration_function based on data or an equation

# Time stepping solution
hook_POOH, time_POOH = dFdt(F_b=[0], operation="POOH")
hook_RIH, time_RIH = dFdt(F_b=[0], operation="RIH")
hook_ROB, time_ROB = dFdt(F_b=[0], operation="ROB")
# hookload = [sol.y.flatten()[-1] for sol in solution]

# print("Hookload values over time:", hookload_over_time)

In [60]:
hook_POOH, time_POOH = dFdt(F_b=[0], operation="POOH")

In [61]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=time_POOH,
    y=hook_POOH,
    mode="lines",
    line=dict(color="blue")
))

# fig.update_layout()
fig.show()

In [13]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=time_POOH,
    y=hook_POOH,
    mode="lines",
    line=dict(color="blue")
))

# fig.update_layout()
fig.show()

In [14]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=time_RIH,
    y=hook_RIH,
    mode="lines",
    line=dict(color="blue")
))

# fig.update_layout()
fig.show()

In [15]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=time_ROB,
    y=hook_ROB,
    mode="lines",
    line=dict(color="blue")
))

# fig.update_layout()
fig.show()

In [ ]:
# MG

End Of NEW VERSION

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Constants
n_elements = 100  # Number of elements in the drillpipe
L = 1000.0  # Total length of the drillpipe in feet
dx = L / n_elements  # Length of each segment
g = 32.2  # Acceleration due to gravity in ft/s^2
mu = 0.2  # Friction coefficient
rho = 0.284  # Density of steel in lb/in^3
A = 5.067  # Cross-sectional area of the drillpipe in in^2 (example for 5" pipe)

# Initial condition for axial forces (hypothetically zero)
initial_forces = np.zeros(n_elements)

# Define the differential equation for axial forces using vectorized operations
def axial_force_eq(t, F):
    dFdt = np.zeros_like(F)
    # Vectorized calculation of the derivative
    dFdt[1:-1] = (F[:-2] - 2 * F[1:-1] + F[2:]) / dx**2
    # Include effects of friction and weight, vectorized
    weight = rho * A * dx * g
    friction = mu * weight
    dFdt[1:-1] -= friction

    # Apply boundary conditions: zero change at the ends as an example
    dFdt[0] = dFdt[-1] = 0
    return dFdt

# Time span for the simulation
t_span = (0, 100)  # Simulate for 10 seconds
t_eval = np.linspace(t_span[0], t_span[1], 300)  # Time points to evaluate the solution

# Solve the ODE using RK45
solution = solve_ivp(axial_force_eq, t_span, initial_forces, method='RK45', t_eval=t_eval)

# Plotting the results
plt.figure(figsize=(10, 5))
plt.plot(t_eval, solution.y.T)
plt.xlabel('Time (seconds)')
plt.ylabel('Axial Force (lbf)')
plt.title('Axial Force Distribution in the Drillpipe Over Time')
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

# Constants and Parameters
n_elements = 5
L = 1000
ds = L / n_elements
w_bp = np.ones(n_elements) * 10
alpha = np.array([0, 45, 45, 90, 90])  # angles in degrees, converted to radians later
mu = np.ones(n_elements) * 0.2
k = np.array([0, 2, 0, 2, 0])
n_z = np.array([0, 1, 0, 1, 0])
b_z = np.array([0, 1, 1, 1, 1])
rho = np.ones(n_elements) * 65.5
A = np.ones(n_elements) * 3.75

# Convert angles to radians for computation
phi = np.deg2rad(alpha)

# Velocity and acceleration setup
v_t = np.array([0, 1.0, 3.0, 3.0])  # Only four relevant velocity points for 20 seconds
times_given = np.array([0, 5, 10, 15])  # Corresponding times

# Create time grid for the simulation
time_steps = np.linspace(0, 20, 200)  # More points may give a smoother result

# Interpolating velocity and acceleration
velocity_interp = interp1d(times_given, v_t, kind='cubic', fill_value="extrapolate", bounds_error=False)
dv_dt_interp = interp1d(times_given, np.gradient(v_t, times_given), kind='cubic', fill_value="extrapolate", bounds_error=False)

# Differential equation for axial force
def axial_force(t, F):
    v_t_now = velocity_interp(t)
    dv_dt_now = dv_dt_interp(t)
    dFdt = np.zeros_like(F)
    
    # Backward mode calculation for axial force
    for i in range(n_elements - 1, 0, -1):
        cos_phi = np.cos(phi[i])
        term1 = w_bp[i] * cos_phi
        term2 = mu[i] * np.sqrt((F[i-1] * k[i] + w_bp[i] * n_z[i] - rho[i] * A[i] * k[i] * v_t_now**2)**2 + (w_bp[i] * b_z[i])**2)
        dFdt[i] = (F[i-1] - F[i]) / ds - term1 - term2 + rho[i] * A[i] * dv_dt_now

    # Set boundary condition at the surface explicitly if needed
    dFdt[0] = 0

    return dFdt

# Initial conditions (zero forces along the pipe)
initial_forces = np.zeros(n_elements)

# Solve the ODE
solution = solve_ivp(axial_force, [0, 20], initial_forces, method='RK45', t_eval=time_steps)

# Plotting the results
plt.figure(figsize=(10, 5))
plt.plot(solution.t, solution.y[0], label='Axial Force at Top')
plt.xlabel('Time (seconds)')
plt.ylabel('Axial Force (lbf)')
plt.title('Axial Force Distribution at the Top of the Pipe Over Time')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
dv_dt

In [ ]:
def p_drop1(rho, mu_p, tao, Q, D_o, D_i, D_w, del_L):
    
    # Annular section
    P_f_an = []
    Area_an = 2.448*(D_w**2-D_o**2)
    v_an = Q/Area_an
    mu_a_an = mu_p + 5*tao*(D_w-D_o)/v_an
    N_re_an = 757*rho*v_an*(D_w-D_o)/mu_a_an
    
    for i in range(len(D_o)):
        if N_re_an[i] > 2100:
            P_f_an.append((rho**0.75*v_an[i]**1.75*mu_p**0.25)/(1396*(D_w[i]-D_o[i])**1.25)) # Turbulent flow case
        else:
            P_f_an.append(((mu_p*v_an[i])/(1000*(D_w[i]-D_o[i])**2) + tao/(200*(D_w[i]-D_o[i])))) # Laminar flow case

    # Inner section
    P_f_in = []
    Area_in = 2.448*D_i**2
    v_in = Q/Area_in
    mu_a_in = mu_p + 6.66*tao*D_i/v_in
    N_re_in = 928*rho*v_in*D_i/mu_a_in
    for i in range(len(D_i)):
        if N_re_in[i] > 2100:
            P_f_in.append((rho**0.75*v_an[i]**1.75*mu_p**0.25)/(1800*D_i[i]**1.25)) # Turbulent flow case
        else:
            P_f_in.append(((mu_p*v_an[i])/(1500*(D_i[i])**2) + tao/(225*D_i[i]))) # Laminar flow case     
    
    return P_f_in, P_f_an, v_in, v_an

In [ ]:
import numpy as np

# All units converted to SI
D = 2.5*0.0254
D1 = 3.5*0.0254
D2 = 5.5*0.0254
Q = 25/264.172/60
rho = 10*119.83
tao_y = 5*0.4788
m = 0.7
K = 10/1000

Annular section begins ::::

In [ ]:
def pressure_gradient1(rho, mu_p, tao, Q, D_o, D_i, D_w):
    # Conversion to SI
    D_i *= 0.0254 # in to m
    D_o *= 0.0254 # in to m
    D_w *= 0.0254 # in to m
    Q /= (264.172*60)  # Gallons per minute to cubic meters per second
    rho *= 119.83  # lbm/ft^3 to kg/m^3
    tao *= 0.4788  # psi to Pa
    mu_p /= 1000  # cp to Pa.s
    m = 1
    
    # Calculation part
    D_hy = D_w - D_o  # Hydraulic diameter
    Area_an = np.pi / 4 * (D_w**2 - D_o**2)
    Area_in = np.pi / 4 * D_i**2
    v_in = Q/Area_in
    v_an = Q/Area_an
    Re_a = (12**(1-m) * D_hy**m * v_an**(2-m) * rho) / (mu_p * ((2*m+1)/(3*m))**m * (1+((2*m+1)/(m+1)) * ((D_hy/(4*v_an)) * (m/(2*m+1)))**m * (tao/mu_p)))
    # Re_a = rho*v_an*D_hy / (mu_p*(1+(D_hy/(8*v_an))*(tao/mu_p)))

    a = np.where(Re_a < 2100, 24, (np.log10(m) + 3.93) / 50)
    b = np.where(Re_a < 2100, 1, (1.75 - np.log10(m)) / 7)

    fric_an = a / (Re_a**b)
    P_f = 2 * fric_an * rho * v_an**2 / D_hy / 22620.60367  # Converting back to Field units (psi/ft)

    return P_f, v_in*3.28084, v_an*3.28084

In [ ]:
def P_an(rho, mu_p, tao, Q, D_o, D_i, D_w):
    # Conversion to SI
    D_i *= 0.0254
    D_o *= 0.0254
    D_w *= 0.0254
    Q /= 264.172*60
    rho *= 119.83
    tao *= 0.4788
    m = 1
    mu_p /= 1000

    # Calculation part
    P_f_an = []
    Area = np.pi / 4 * (D_w**2 - D_o**2)
    v_an = Q / Area
    gamma_an = 12 * v_an / (D_w - D_o)
    t_initial = tao + mu_p * gamma_an**m
    # tao_w_eff = t_initial.copy()  # Initial effective wall shear stress
    
    for i in range(len(D_o)):
        tol = 1
        tao_w_eff = [t_initial[i]]

        while tol>=0.0001:
            x = tao/tao_w_eff[-1]
            Cc = (1-x)*(m*x/(m+1) + 1)
            D_e = (3*m/(2*m + 1))*Cc*(D_w[i] - D_o[i])
            gamma_e = 12*v_an[i]/D_e
            t_final = tao + mu_p*gamma_e**m
            tol = abs((t_final - tao_w_eff[-1])/t_final)
            tao_w_eff.append(t_final)

        N_re = 12 * rho * v_an[i]**2 / tao_w_eff[-1]
        N = np.log(tao_w_eff[-1]) / np.log(12 * v_an[i] / (D_w[i] - D_o[i]))
        N_crit = 3250 - 1150 * N
        if N_crit > N_re:
            fric_an = 24/N_re
            P_f_an.append(2*fric_an*rho*v_an[i]**2/(D_w[i] - D_o[i])/22620.60367) # Converting back to Field units (psi/ft)
        else:
            P_f_an.append(777)

    return P_f_an, v_an


In [ ]:
P_f_an = []
Area_an = np.pi/4*(D2**2-D1**2)
v_an = Q/Area_an

# Iterations
tol = 1
gamma_an = 12*v_an/(D2 - D1)
t_initial_an = tao_y + K*gamma_an**m
tao_w_eff_an = [t_initial_an]

while tol>=0.0001:
    x = tao_y/tao_w_eff_an[-1]
    Cc = (1-x)*(m*x/(m+1) + 1)
    D_e = (3*m/(2*m + 1))*Cc*(D2-D1)
    gamma_e = 12*v_an/D_e
    t_final = tao_y + K*gamma_e**m
    tol = abs((t_final - tao_w_eff_an[-1])/t_final)
    tao_w_eff_an.append(t_final)

N_re_an = 12*rho*v_an**2/tao_w_eff_an[-1]
N_an = np.log(tao_w_eff_an[-1])/np.log(12*v_an/(D2 - D1))
N_crit_an = 3250 - 1150*N_an

if N_crit_an > N_re_an:
    fric_an = 24/N_re_an
    P_f_an.append(2*fric_an*rho*v_an**2/(D2 - D1)/22620.60367) # Converting back to Field units (psi/ft)
else:
    P_f_an.append(777)
    

Inside section begins ::::

In [ ]:
P_f_in = []
Area_in = np.pi/4*D**2
v_in = Q/Area_in
# Iterations
tol = 1
gamma_in = 8*v_in/D
t_initial_in = tao_y + K*gamma_in**m
tao_w_eff_in = [t_initial_in]

while tol>=0.0001:
    x = tao_y/tao_w_eff_in[-1]
    Cc = (1-x)*(2*m**2*x**2/((1+2*m)*(1+m)) + 2*m*x/(1+2*m) + 1)
    D_e = (4*m/(3*m + 1))*Cc*(D)
    gamma_e = 8*v_in/D_e
    t_final = tao_y + K*gamma_e**m
    tol = abs((t_final - tao_w_eff_in[-1])/t_final)
    tao_w_eff_in.append(t_final)

N_re_in = 8*rho*v_in**2/tao_w_eff_in[-1]
N_in = np.log(tao_w_eff_in[-1])/np.log(8*v_in/D)
N_crit_in = 3250 - 1150*N_in

if N_crit_in > N_re_in:
    fric_in = 16/N_re_in
    P_f_in.append(2*fric_in*rho*v_in**2/D/22620.60367) # Converting back to Field units (psi/ft)
else:
    P_f_in.append(999)

In [ ]:
import numpy as np

inc = np.array([0,0,45,45,90])
azi = np.array([0,15,30,30,45])
inc_rad = np.radians(inc)
azi_rad = np.radians(azi)
s = np.array([0,45,150,200,650])

dinc = np.diff(inc_rad)
dazi = np.diff(azi_rad)
ds = np.diff(s)

# Calculate the average of the inc and azi values at the intervals where the differences are calculated
inc_rad_avg = (inc_rad[:-1] + inc_rad[1:]) / 2
azi_rad_avg = (azi_rad[:-1] + azi_rad[1:]) / 2

t = np.column_stack([
    np.sin(inc_rad) * np.cos(azi_rad),
    np.sin(inc_rad) * np.sin(azi_rad),
    np.cos(inc_rad)
])

K = np.column_stack([
    np.cos(inc_rad_avg) * np.cos(azi_rad_avg) * (dinc / ds) - np.sin(inc_rad_avg) * np.sin(azi_rad_avg) * (dazi / ds),
    np.cos(inc_rad_avg) * np.sin(azi_rad_avg) * (dinc / ds) + np.sin(inc_rad_avg) * np.cos(azi_rad_avg) * (dazi / ds),
    -np.sin(inc_rad_avg) * (dinc / ds)
])

In [ ]:
from lib.constants import *
import numpy as np

In [ ]:
clc = Calculations(depth=13000)
P_drop_inner , P_drop_outer, v_i, v_o = p_drop1(
    Q=250,
    rho=clc.mud_density_ppg, 
    mu_p=clc.visc_p, 
    tao=clc.tao_y,
    D_o=clc.global_od_array, 
    D_i=clc.global_id_array,
    D_w=clc.global_hole_array,
    del_L=clc.global_length_array
)

In [ ]:
clc = Calculations(depth=13000)
P_annl, v_in, v_annl = pressure_gradient1(
    Q=250,
    rho=clc.mud_density_ppg, 
    mu_p=clc.visc_p, 
    tao=clc.tao_y,
    D_o=clc.global_od_array, 
    D_i=clc.global_id_array,
    D_w=clc.global_hole_array
)

In [ ]:
len(P_drop_outer), len(P_annl)

In [ ]:
sum(P_drop_outer)/len(P_drop_outer), sum(P_annl)/len(P_annl)

In [ ]:
sum(P_drop_outer), sum(P_annl)

In [ ]:
sum(v_o)/len(v_o), sum(v_annl)/len(v_annl)

In [ ]:
sum(v_i)/len(v_i), sum(v_in)/len(v_in)

In [ ]:
P_annl

In [ ]:
P_drop_outer

Effective tension

In [ ]:
# Area_o = np.pi*(clc.global_hole_array**2 - clc.global_od_array**2)/4
Area_o = np.pi*(clc.global_od_array**2)/4
Area_i = np.pi*(clc.global_id_array**2)/4
C_tao_0 = (8 + 4*clc.global_eps + clc.global_eps**2)/8


Static_part = (12/231)*(Area_i-Area_o)*clc.mud_density_ppg*clc.gravity*clc.t_z
# Annular_part = (P_drop_outer*Area_o*clc.t_z_min)*(1+C_tao_0)
# Annular_part = (1+C_tao_0)*P_drop_outer*Area_o
Annular_part = (1+C_tao_0)*P_drop_outer*Area_o
Inertial_part = (12/231)*clc.mud_density_ppg*(v_o**2*Area_o - v_i**2*Area_i)*clc.DLS*clc.n_z

lin_force = -Annular_part + Inertial_part
fluid_force = np.flip(lin_force)
force_ave = fluid_force.sum()/len(fluid_force)
# total_force = np.cumsum(fluid_force)


In [ ]:
fluid_force.sum()/len(fluid_force)

In [ ]:
Static_part.sum()/len(Static_part)

In [ ]:
Inertial_part.sum()/len(Inertial_part)

In [ ]:
Annular_part.sum()/len(Annular_part)

In [ ]:
from lib.constants import Calculations
from scipy.integrate import solve_ivp
import numpy as np

def trip_operation(F_b, ff, flowrate, length, operation='POOH'):
    clc = Calculations(lengths=length)
    new_force = fluid_force if flowrate != 0 else np.zeros_like(clc.MD[:-1])
    def dFds(s, F):
        # Interpolate the necessary values for the current s
        DLS_interp = np.interp(s, clc.MD[:-1], clc.DLS)
        n_z_interp = np.interp(s, clc.MD[:-1], clc.n_z)
        b_z_interp = np.interp(s, clc.MD[:-1], clc.b_z)
        t_z_interp = np.interp(s, clc.MD[:-1], clc.t_z)
        bw_pipe_interp = np.interp(s, clc.MD[:-1], clc.bw_pipe)
        fluid_interp = np.interp(s, clc.MD[:-1], new_force)

        # Calculate the contact force
        w_c = np.sqrt((F * DLS_interp + bw_pipe_interp * n_z_interp)**2 + (bw_pipe_interp * b_z_interp)**2)

        if operation == 'POOH':
            return -(bw_pipe_interp*t_z_interp + ff*w_c + fluid_interp)
        elif operation == 'RIH':
            return -(bw_pipe_interp*t_z_interp - ff*w_c + fluid_interp)
        elif operation == 'ROB':
            return -(bw_pipe_interp*t_z_interp + fluid_interp)
        else:
            raise ValueError("Unknown operation type specified.")

    s_span = [clc.MD[-1], clc.MD[0]]  # Span from the end to the start
    solution = solve_ivp(dFds,
                         s_span,
                         [F_b],
                         method='RK45',
                         t_eval=np.flip(clc.MD),
                         max_step=5,
                         rtol=0.001,
                         atol=0.001,
                         dense_output=True
    ) 
    
    return solution.y.flatten()

In [ ]:
ROB470 = trip_operation(F_b=0, ff=0.15, flowrate=470, length=13000, operation='ROB')[-1]
ROB250 = trip_operation(F_b=0, ff=0.15, flowrate=250, length=13000, operation='ROB')[-1]
ROB0 = trip_operation(F_b=0, ff=0.15, flowrate=0, length=17000, operation='ROB')[-1]

In [ ]:
ROB470, ROB250, ROB0

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig1 = make_subplots(rows=1, cols=1)

fig1.add_trace(go.Scatter(x=fluid_force, y=np.linspace(13000,0,len(fluid_force)), name='13k @ 470 gpm', line=dict(color='darkred', width=3, dash='dashdot')), row=1, col=1)

# Update the layout

fig1.update_layout(
    
    template= "none",
    title='Fluid Force for Various GPM Rates',
    title_font_size=24,
    xaxis=dict(
        title='Fluid Force [lbf]',
        titlefont_size=20
    ),
    yaxis=dict(
        title='Bit Depth [ft]',
        titlefont_size=20,
        autorange="reversed"
    ),
    legend=dict(
        traceorder='normal',
        font=dict(
            size=16,
        )
    ),
    height = 600,
    width = 1000
)

fig1.show()
# fig.write_html("Plot.html")

In [ ]:
clc.DLS.tolist().index(0.008118177518585505)

In [ ]:
clc.DLS[360:365]

In [ ]:
clc.MD[360:370]

In [ ]:
newMD = np.flip(clc.MD).tolist()

In [ ]:
newMD[1], newMD[39], newMD[-65]

In [ ]:
Annular_part = np.flip(Annular_part)

In [ ]:
Inertial_part = np.flip(Inertial_part)

In [ ]:
Annular_part

In [ ]:
Annular_part[1]

In [ ]:
Annular_part[39]

In [ ]:
Annular_part[-65]

In [ ]:
Inertial_part[1]

In [ ]:
Inertial_part[39]

In [ ]:
Inertial_part[-65]

In [ ]:
flu_250_13k = fluid_force



In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig1 = make_subplots(rows=1, cols=1)

fig1.add_trace(go.Scatter(x=flu_250_13k, y=np.linspace(13000,0,len(flu_250_13k)), name='13k @ 470 gpm', line=dict(color='darkred', width=3, dash='dashdot')), row=1, col=1)

# Update the layout

fig1.update_layout(
    
    template= "none",
    title='Fluid Force for Various GPM Rates',
    title_font_size=24,
    xaxis=dict(
        title='Fluid Force [lbf/ft]',
        titlefont_size=20
    ),
    yaxis=dict(
        title='MD [ft]',
        titlefont_size=20,
        autorange="reversed"
    ),
    legend=dict(
        traceorder='normal',
        font=dict(
            size=16,
        )
    ),
    height = 600,
    width = 1000
)

fig1.show()
# fig.write_html("Plot.html")

In [ ]:
[trip_operation(F_b=0, ff=0.15, length=i, operation='POOH')[-1] for i in np.arange(12000, 18000, 100)]

In [ ]:
trip_operation(F_b=0, ff=0.15, length=13000, operation='RIH')

In [ ]:
trip_operation(F_b=0, ff=0.15, length=13000, operation='ROB')